In [26]:
import numpy as np
import json
from graphviz import Digraph

In [27]:
# Activation models enum
activation = [
  'relu',
  'sigmoid',
  'linear',
  'softmax'
]

In [28]:
## Class layer

class Layer:
  def __init__(self, neuron: int, function: str, weights: np.array, bias: np.array):
    self.neuron = neuron
    self.weights = weights
    self.bias = bias
    if (function not in activation):
      raise Exception('Activation function not found')
    else:
      self.str_function = function
      if self.str_function == 'sigmoid':
        self.function = lambda x: 1 / (1 + np.exp(-x))
      elif self.str_function == 'relu':
        self.function = lambda x: np.maximum(0, x)
      elif self.str_function == 'linear':
        self.function = lambda x: x
      else:
        self.function = lambda x: np.exp(x) / np.sum(np.exp(x))

  def forward(self, input: np.array):
    self.output = self.function(np.dot(input, self.weights) + self.bias)
    return self.output

In [29]:
class FFNN:
  def __init__ (self, input: np.array, layers: list):
    self.input = input
    self.layers = layers
    self.output = None
  
  def new_layer(self, layer: Layer):
    self.layers.append(layer)

  def forward(self):
    self.output = self.input
    for layer in self.layers:
      self.output = layer.forward(self.output)

    return self.output
  
  def predictions(self):
    if(len(self.output.shape) == 1):
      for i in range(len(self.output)):
        if self.output[i] > 0.5:
          self.output[i] = 1
        else:
          self.output[i] = 0
    else :
      for i in range(len(self.output)):
        for j in range(len(self.output[i])):
          if self.output[i][j] > 0.5:
            self.output[i][j] = 1
          else:
            self.output[i][j] = 0

  def visualize(self):
    dot = Digraph(comment='FFNN')
    dot.attr(rankdir='LR', nodesep='1', ranksep='')
    dot.attr('node', shape='circle', width='0.4', height='0.4')
    
    #if no hidden layer
    if len(self.layers) == 1:
      for i in range(len(self.input[0])):
        for j in range(len(self.output[0])):
          weight = self.layers[0].weights[i][j]
          dot.edge(f'input{i}', f'output{j}', xlabel=f'{weight:.2f}', color='#2ecc71', xlabelfloat='true')
    else :
    # Input layer
      for i in range(len(self.input[0])):
          dot.node(f'input{i}', f'input{i}', color='#2ecc71')

      # Hidden layers
      for i in range(len(self.layers) - 1):
          for j in range(self.layers[i+1].neuron):
              dot.node(f'hidden{i}{j}', f'hidden{i}{j}', color='#e67e22')

          if i == 0:
              for j in range(len(self.input[0])):
                  for k in range(self.layers[i+1].neuron):
                      weight = self.layers[i].weights[j][k]
                      dot.edge(f'input{j}', f'hidden{i}{k}', xlabel=f'{weight:.2f}', color='#2ecc71')

          else:
              for j in range(self.layers[i].neuron):
                  for k in range(self.layers[i+1].neuron):
                      weight = self.layers[i].weights[j][k]
                      dot.edge(f'hidden{i-1}{j}', f'hidden{i}{k}', xlabel=f'{weight:.2f}', color='#e67e22')

      # Output layer
      for i in range(len(self.output[0])):
          dot.node(f'output{i}', f'output{i}', color='#f1c40f')

      for i in range(self.layers[-1].neuron):
          for j in range(len(self.output[0])):
              weight = self.layers[-1].weights[i][j]
              dot.edge(f'hidden{len(self.layers)-2}{i}', f'output{j}', xlabel=f'{weight:.2f}', color='#f1c40f')


    return dot

In [43]:
model = str(input("Masukkan nama file model (dalam format .json): "))
model = open(f"models/{model}", "r")
model = json.load(model)

layers = model["case"]["model"]
weights = model["case"]["weights"]
inputArray = model["case"]["input"]

ffnn = FFNN(np.array(inputArray), [])
for i in range (len(layers["layers"])):
  layer = layers["layers"][i]
  weight = weights[i]
  new_layer = Layer(layer["number_of_neurons"], layer["activation_function"], np.array(weight[1:]), np.array(weight[0]))
  ffnn.new_layer(new_layer)

ffnn.forward()


print(ffnn.output)

[[0.66524096 0.09003057 0.24472847]]


In [44]:
# Create Graphviz digraph object
dot = ffnn.visualize()


# Save and render the graph
dot.render("ffnn_graph", format="png", cleanup=True)

IndexError: index 2 is out of bounds for axis 0 with size 2